<a href="https://colab.research.google.com/github/bdunn20/book_code/blob/main/pandas_1.X_cookbook/Ch9_Grouping_for_Aggregation_Filtration_and_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chapter 9: Groupging for Aggregation, Filtration, and Transformation



### Defining an aggregation

In [2]:
import pandas as pd
import numpy as np

In [3]:
flights = pd.read_csv("https://raw.githubusercontent.com/PacktPublishing/Pandas-Cookbook-Second-Edition/master/data/flights.csv")
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [4]:
# define the grouping columns (AIRLINE), aggregating columns (ARR_DELAY), and
# aggregating functions (mean)

flights.groupby('AIRLINE').agg({'ARR_DELAY':'mean'})

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580
F9,13.630651
HA,4.972973
MQ,6.860591
NK,18.436070


### Grouping & aggregating w/ multiple columns and functions.

In [5]:
# find the number of canceled flights for every airline per weekday
flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED'].agg('sum')

AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
                    ..
WN       3          18
         4          10
         5           7
         6          10
         7           7
Name: CANCELLED, Length: 98, dtype: int64

In [6]:
# find the number and percentage of canceled and diverted flights for every airline per weekday
flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED', 'DIVERTED'].agg(['sum', 'mean'])

<ipython-input-6-71ce9411a89a>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED', 'DIVERTED'].agg(['sum', 'mean'])


CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786
...                   ...       ...      ...       ...
WN      3              18  0.014118        2  0.001569
        4              10  0.007911        4  0.003165
        5               7  0.005828        0  0.000000
        6              10  0.010132        3  0.003040
        7               7  0.006066        3  0.002600

[98 rows x 4 columns]

In [7]:
# for each origin & destination, find the total number of flights, the number and
# percentage of canceled flights, and the avg and variance of the airtime
flights.groupby(['ORG_AIR', 'DEST_AIR']).agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
ATL     ABE              0  0.000000   31   96.387097   45.778495
        ABQ              0  0.000000   16  170.500000   87.866667
        ABY              0  0.000000   19   28.578947    6.590643
        ACY              0  0.000000    6   91.333333   11.466667
        AEX              0  0.000000   40   78.725000   47.332692
...                    ...       ...  ...         ...         ...
SFO     SNA              4  0.032787  122   64.059322   11.338331
        STL              0  0.000000   20  198.900000  101.042105
        SUN              0  0.000000   10   78.000000   25.777778
        TUS              0  0.000000   20  100.200000   35.221053
        XNA              0  0.000000    2  173.500000    0.500000

[1130 rows x 5 columns]

### Removing the MultiIndex after grouping

In [8]:
# find the total and avg miles flown and the min/max arrival delay for each airline for each weekday
airline_info = flights.groupby(['AIRLINE', 'WEEKDAY']).agg({'DIST':['sum', 'mean'], 'ARR_DELAY':['min', 'max']}).astype(int)

In [9]:
airline_info

DIST       ARR_DELAY     
                     sum  mean       min  max
AIRLINE WEEKDAY                              
AA      1        1455386  1139       -60  551
        2        1358256  1107       -52  725
        3        1496665  1117       -45  473
        4        1452394  1089       -46  349
        5        1427749  1122       -41  732
...                  ...   ...       ...  ...
WN      3         997213   782       -38  262
        4        1024854   810       -52  284
        5         981036   816       -44  244
        6         823946   834       -41  290
        7         945679   819       -45  261

[98 rows x 4 columns]

In [10]:
# both rows and columns are now labeled by a MultiIndex
# to address the columns, we use the MultiIndex method .to_flat_index
airline_info.columns.get_level_values(0)

Index(['DIST', 'DIST', 'ARR_DELAY', 'ARR_DELAY'], dtype='object')

In [11]:
airline_info.columns.get_level_values(1)

Index(['sum', 'mean', 'min', 'max'], dtype='object')

In [12]:
airline_info.columns.to_flat_index()

Index([('DIST', 'sum'), ('DIST', 'mean'), ('ARR_DELAY', 'min'),
       ('ARR_DELAY', 'max')],
      dtype='object')

In [13]:
airline_info.columns = ['_'.join(x) for x in airline_info.columns.to_flat_index()]
airline_info

DIST_sum  DIST_mean  ARR_DELAY_min  ARR_DELAY_max
AIRLINE WEEKDAY                                                   
AA      1         1455386       1139            -60            551
        2         1358256       1107            -52            725
        3         1496665       1117            -45            473
        4         1452394       1089            -46            349
        5         1427749       1122            -41            732
...                   ...        ...            ...            ...
WN      3          997213        782            -38            262
        4         1024854        810            -52            284
        5          981036        816            -44            244
        6          823946        834            -41            290
        7          945679        819            -45            261

[98 rows x 4 columns]

In [14]:
# quick way to get rid of the row MultiIndex is to use the .reset_index method
airline_info.reset_index()

,AIRLINE,WEEKDAY,DIST_sum,DIST_mean,ARR_DELAY_min,ARR_DELAY_max
0,AA,1,1455386,1139,-60,551
1,AA,2,1358256,1107,-52,725
2,AA,3,1496665,1117,-45,473
3,AA,4,1452394,1089,-46,349
4,AA,5,1427749,1122,-41,732
...,...,...,...,...,...,...
93,WN,3,997213,782,-38,262
94,WN,4,1024854,810,-52,284
95,WN,5,981036,816,-44,244
96,WN,6,823946,834,-41,290


### Grouping with a custom aggregation function

In [15]:
college = pd.read_csv("https://raw.githubusercontent.com/PacktPublishing/Pandas-Cookbook-Second-Edition/master/data/college.csv?_sm_au_=iVVD4PQ5PHvFk37FBQvGvK7jj80tt")

college.groupby('STABBR')['UGDS'].agg(['mean', 'std']).round(0)

,mean,std
STABBR,,
AK,2493.0,4052.0
AL,2790.0,4658.0
AR,1644.0,3143.0
AS,1276.0,NaN
AZ,4130.0,14894.0
CA,3518.0,6709.0
CO,2325.0,4670.0
CT,1874.0,2871.0
DC,2645.0,3225.0


In [16]:
# the above is the mean & std of the entire group, which is not what we're looking for
# we're looking for the max number of std from the mean for any one institution
# to calc this, we need to subtract the mean undergrad population by state from each
# institution's undergrad population and then divide by std.
# this standardizes the undergrad population for each group
# then we can take the max of absolute value of these scores to find the one
# farthest away from the mean
# pandas does not provide a function capable so we create a custom function
def max_deviation(s):
  std_score = (s - s.mean()) / s.std()
  return std_score.abs().max()

In [17]:
# after definining the function, pass directly to .agg method to complete
college.groupby('STABBR')['UGDS'].agg(max_deviation).round(1)

STABBR
AK     2.6
AL     5.8
AR     6.3
AS     NaN
AZ     9.9
CA     6.1
CO     5.0
CT     5.6
DC     2.4
DE     3.5
FL     8.4
FM     NaN
GA     5.4
GU     1.0
HI     3.8
IA     6.5
ID     4.5
IL     7.3
IN     9.1
KS     4.9
KY     5.2
LA     6.5
MA     6.1
MD     5.3
ME     4.0
MH     NaN
MI     6.7
MN     7.8
MO     7.2
MP     NaN
MS     4.0
MT     3.9
NC     4.9
ND     3.5
NE     5.0
NH     5.3
NJ     7.1
NM     4.5
NV     4.7
NY     8.2
OH    10.3
OK     5.9
OR     5.3
PA    10.1
PR     6.0
PW     NaN
RI     2.9
SC     6.0
SD     4.2
TN     6.0
TX     7.7
UT     5.1
VA     7.0
VI     NaN
VT     3.8
WA     6.6
WI     5.8
WV     7.2
WY     2.8
Name: UGDS, dtype: float64

### Customizing aggregating functions with *args and **kwargs

In [18]:
# define a function that returns the % of schools w/ undergrad population b/t 1k-3k
def pct_between_1_3k(s):
  return(s
         .between(1_000, 3_000)
         .mean()
         *100
         )

In [19]:
# calc % grouping by state and religious affiliation
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(pct_between_1_3k).round(1)

STABBR  RELAFFIL
AK      0           14.3
        1            0.0
AL      0           23.6
        1           33.3
AR      0           27.9
                    ... 
WI      0           13.8
        1           36.0
WV      0           24.6
        1           37.5
WY      0           54.5
Name: UGDS, Length: 112, dtype: float64

In [20]:
# create new function that allows the user to parameterize the lower & upper bound
def pct_between(s, low, high):
  return s.between(low, high).mean() * 100

In [21]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(pct_between, 1_000, 10_000).round(1)

STABBR  RELAFFIL
AK      0           42.9
        1            0.0
AL      0           45.8
        1           37.5
AR      0           39.7
                    ... 
WI      0           31.0
        1           44.0
WV      0           29.2
        1           37.5
WY      0           72.7
Name: UGDS, Length: 112, dtype: float64

### Examining the groupby object

In [22]:
# group the state & religious affiliation columns, saving the result to a variable
grouped = college.groupby(['STABBR', 'RELAFFIL'])
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

In [23]:
print([attr for attr in dir(grouped) if not attr.startswith('_')])

['CITY', 'CURROPER', 'DISTANCEONLY', 'GRAD_DEBT_MDN_SUPP', 'HBCU', 'INSTNM', 'MD_EARN_WNE_P10', 'MENONLY', 'PCTFLOAN', 'PCTPELL', 'PPTUG_EF', 'RELAFFIL', 'SATMTMID', 'SATVRMID', 'STABBR', 'UG25ABV', 'UGDS', 'UGDS_2MOR', 'UGDS_AIAN', 'UGDS_ASIAN', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_NHPI', 'UGDS_NRA', 'UGDS_UNKN', 'UGDS_WHITE', 'WOMENONLY', 'agg', 'aggregate', 'all', 'any', 'apply', 'backfill', 'bfill', 'boxplot', 'corr', 'corrwith', 'count', 'cov', 'cumcount', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'dtypes', 'ewm', 'expanding', 'ffill', 'fillna', 'filter', 'first', 'get_group', 'groups', 'head', 'hist', 'idxmax', 'idxmin', 'indices', 'last', 'mad', 'max', 'mean', 'median', 'min', 'ndim', 'ngroup', 'ngroups', 'nth', 'nunique', 'ohlc', 'pad', 'pct_change', 'pipe', 'plot', 'prod', 'quantile', 'rank', 'resample', 'rolling', 'sample', 'sem', 'shift', 'size', 'skew', 'std', 'sum', 'tail', 'take', 'transform', 'tshift', 'value_counts', 'var']


In [24]:
# find the number of groups with the .ngroups attribute
grouped.ngroups

112

In [25]:
# find the uniquely identifying labels for each group
groups = list(grouped.groups)
groups[:6]

[('AK', 0), ('AK', 1), ('AL', 0), ('AL', 1), ('AR', 0), ('AR', 1)]

In [26]:
# retrieve a single group w/ the .get_group method
# below retrieves all the religiously affiliated schools in the state of Florida
grouped.get_group(('FL', 1))

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
712,The Baptist College of Florida,Graceville,FL,0.0,0.0,0.0,1,545.0,465.0,0.0,...,0.0308,0.0000,0.0507,0.2291,1,0.5878,0.5602,0.3531,30800,20052
713,Barry University,Miami,FL,0.0,0.0,0.0,1,470.0,462.0,0.0,...,0.0164,0.0741,0.0841,0.1518,1,0.5045,0.6733,0.4361,44100,28250
714,Gooding Institute of Nurse Anesthesia,Panama City,FL,0.0,0.0,0.0,1,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,PrivacySuppressed
715,Bethune-Cookman University,Daytona Beach,FL,1.0,0.0,0.0,1,405.0,395.0,0.0,...,0.0198,0.0205,0.0190,0.0523,1,0.7758,0.8867,0.0647,29400,36250
724,Johnson University Florida,Kissimmee,FL,0.0,0.0,0.0,1,480.0,470.0,0.0,...,0.0045,0.0045,0.0136,0.1636,1,0.6689,0.7384,0.2185,26300,20199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7486,Strayer University-Coral Springs Campus,Coral Springs,FL,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,49200,36173.5
7487,Strayer University-Fort Lauderdale Campus,Fort Lauderdale,FL,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,49200,36173.5
7488,Strayer University-Miramar Campus,Miramar,FL,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,49200,36173.5
7489,Strayer University-Doral,Miami,FL,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,49200,36173.5


In [28]:
# leverage the display function to show each group in a single cell
from IPython.display import display
for name, group in grouped:
    print(name)
    display(group.head(3))

('AK', 0)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
60,University of Alaska Anchorage,Anchorage,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0980,0.0181,0.0457,0.4539,1,0.2385,0.2647,0.4386,42500,19449.5
62,University of Alaska Fairbanks,Fairbanks,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0401,0.0110,0.3060,0.3887,1,0.2263,0.2550,0.4519,36200,19355
63,University of Alaska Southeast,Juneau,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0686,0.0049,0.2241,0.5112,1,0.1769,0.1996,0.5550,37400,16875


('AK', 1)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
61,Alaska Bible College,Palmer,AK,0.0,0.0,0.0,1,NaN,NaN,0.0,...,0.0370,0.0,0.0000,0.1481,1,0.3571,0.2857,0.4286,NaN,PrivacySuppressed
64,Alaska Pacific University,Anchorage,AK,0.0,0.0,0.0,1,555.0,503.0,0.0,...,0.0945,0.0,0.0873,0.3745,1,0.3152,0.5297,0.4910,47000,23250
5417,Alaska Christian College,Soldotna,AK,0.0,0.0,0.0,1,NaN,NaN,0.0,...,0.0147,0.0,0.1324,0.0735,1,0.8868,0.6792,0.2264,NaN,PrivacySuppressed


('AL', 0)


...

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
2842,Scott College of Cosmetology,Wheeling,WV,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0519,0.0,0.0,0.0000,1,0.5276,0.4961,0.1111,14800,9250
4019,B M Spurr School of Practical Nursing,Glen Dale,WV,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0000,0.0,0.0,0.0000,1,0.4722,0.0000,0.4444,PrivacySuppressed,PrivacySuppressed
4020,Ben Franklin Career Center,Dunbar,WV,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0023,0.0,0.0,0.7885,1,0.1761,0.0000,0.7568,20800,PrivacySuppressed


('WV', 1)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
4016,Alderson Broaddus University,Philippi,WV,0.0,0.0,0.0,1,475.0,485.0,0.0,...,0.0098,0.0342,0.0020,0.0342,1,0.4800,0.8945,0.0722,46000,27000
4018,Appalachian Bible College,Mount Hope,WV,0.0,0.0,0.0,1,514.0,487.0,0.0,...,0.0000,0.0051,0.0000,0.0612,1,0.4388,0.2770,0.0899,28700,9300
4027,Davis & Elkins College,Elkins,WV,0.0,0.0,0.0,1,485.0,490.0,0.0,...,0.0000,0.1466,0.1657,0.0167,1,0.4188,0.6579,0.1133,35000,23840.5


('WY', 0)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
4128,Casper College,Casper,WY,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0163,0.0078,0.0269,0.3250,1,0.2188,0.1455,0.3447,34800,10764
4129,Central Wyoming College,Riverton,WY,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0300,0.0073,0.0173,0.4377,1,0.1700,0.0967,0.3992,25200,8757
4130,Eastern Wyoming College,Torrington,WY,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0088,0.0015,0.0000,0.3275,1,0.1729,0.1166,0.2371,25900,10000
